In [2]:
import pandas

from matplotlib import pyplot
from deps.data import load_metadata
from deps.logger import logger
from deps.data import load_raw_data, load_data
from hcve_lib.functional import pipe
from hcve_lib.evaluation_functions import c_index
from hcve_lib.data import format_identifier_long
from hcve_lib.visualisation import p
from optuna import Trial
from hcve_lib.tracking import get_completed_runs, get_run_duration
from hcve_lib.tracking import get_last_run
from hcve_lib.tracking import is_root_run
from hcve_lib.tracking import get_children_runs
import humanize
import pytz
from datetime import datetime
from deps.common import get_variables_cached

logger.setLevel('DEBUG')
pandas.set_option("display.max_columns", None)
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina'
pyplot.rcParams['figure.facecolor'] = 'white'

/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/dash_bootstrap_components/_table.py:5: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



In [32]:
data, metadata, X, y = get_variables_cached()

raw_data = load_raw_data()
raw_data_subset = raw_data.loc[data.index]

In [6]:
data = load_data(metadata)
data[data['FUNFHF'].isna()]

Raw data
	n cohorts=21
	n individuals=52,585

[17:01:18] Raw data
	n cohorts=21
	n individuals=52,585

Baseline visit kept
 	n individuals removed=5,529

 	n individuals=47,056

[17:01:18] Baseline visit kept
 	n individuals removed=5,529

 	n individuals=47,056

HF cohorts removed
	n cohorts removed=3: hfgr, timechf, leitzaran
	n individuals removed=1,073

	n cohorts=18
	n individuals=45,983

[17:01:18] HF cohorts removed
	n cohorts removed=3: hfgr, timechf, leitzaran
	n individuals removed=1,073

	n cohorts=18
	n individuals=45,983

No outcome cohorts removed
	n cohorts removed=5: biomarcoeurs, dyda, epath, iblomaved, stophf
	n individuals removed=3,077

	n cohorts=13
	n individuals=42,906

[17:01:18] No outcome cohorts removed
	n cohorts removed=5: biomarcoeurs, dyda, epath, iblomaved, stophf
	n individuals removed=3,077

	n cohorts=13
	n individuals=42,906

Missing HF data cohorts removed
	n cohorts removed=6: adelhyde, gecoh, r2c2, reve(1-2), stanislas, styrianvitd
	n individuals 

,IDNR,STUDY_NUM,STUDY,VISIT,SEX,AGE,BW,BH,BMI,SMK,DRK,DIABETES,HCV,HCAD,HPAD,HAF,HCEREBROV,TRT_AH,TRT_ACE,TRT_CCB,TRT_DIUR,TRT_BB,TRT_ARB,TRT_LIP,SBP,DBP,HR,PP,SOK,QRS,CI,GLU,CHOL,LDL,HDL,TG,CREA,FCV,FUFCV,NFHF,FUNFHF,FNFHF,FUFNFHF
IDNR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [7]:
for column in data.columns:
    print(f'{len(data[column].unique())}: {format_identifier_long(column, metadata)}')

32405: [IDNR] IDNR
7: [STUDY_NUM] Numeric study identifier
7: [STUDY] STUDY
1: [VISIT] VISIT
2: [SEX] Gender
16090: [AGE] Age
1878: [BW] Body weight
883: [BH] Body height
13216: [BMI] Body-mass index_frame
4: [SMK] Smoking status
3: [DRK] Alcohol intake
3: [DIABETES] History of diabetes
3: [HCV] History of cardiovascular disease
3: [HCAD] History of coronary artery disease
3: [HPAD] History of peripheral artery disease
3: [HAF] History of atrial fibrillation
3: [HCEREBROV] History of cerebrovascular incident
3: [TRT_AH] Intake of antihypertensive medication
3: [TRT_ACE] Intake of angiotensin converting enzyme inhibitors
3: [TRT_CCB] Intake of calcium channel blockers
3: [TRT_DIUR] Intake of diuretics
3: [TRT_BB] Intake of beta-blokcers
3: [TRT_ARB] Intake of angiotensin II receptor blockers
3: [TRT_LIP] Intake of lipid-lowering medication
471: [SBP] Systolic blood pressure
308: [DBP] Diastolic blood pressure
189: [HR] Heart rate
408: [PP] Pulse pressure
557: [SOK] Sokolow-Lyon index_fr

In [8]:
runs_lm = get_completed_runs('lm')
root_runs = runs_lm[is_root_run(runs_lm)]

int64: [IDNR] IDNR
category: [STUDY_NUM] Numeric study identifier
category: [STUDY] STUDY
category: [VISIT] VISIT
category: [SEX] Gender
float64: [AGE] Age
float64: [BW] Body weight
float64: [BH] Body height
float64: [BMI] Body-mass index_frame
category: [SMK] Smoking status
category: [DRK] Alcohol intake
category: [DIABETES] History of diabetes
category: [HCV] History of cardiovascular disease
category: [HCAD] History of coronary artery disease
category: [HPAD] History of peripheral artery disease
category: [HAF] History of atrial fibrillation
category: [HCEREBROV] History of cerebrovascular incident
category: [TRT_AH] Intake of antihypertensive medication
category: [TRT_ACE] Intake of angiotensin converting enzyme inhibitors
category: [TRT_CCB] Intake of calcium channel blockers
category: [TRT_DIUR] Intake of diuretics
category: [TRT_BB] Intake of beta-blokcers
category: [TRT_ARB] Intake of angiotensin II receptor blockers
category: [TRT_LIP] Intake of lipid-lowering medication
float